In [1]:
from collections import defaultdict, Counter

god_rmse = defaultdict(list)

In [2]:
from mangaki.utils.als import MangakiALS
from mangaki.utils.svd import MangakiSVD

In [3]:
from mangaki.utils.algo import get_dataset_backup
from mangaki.utils.data import Dataset
from mangaki.utils.values import rating_values

dataset = Dataset()
dataset.load_csv('balse/ratings.csv', convert=lambda x: rating_values[x], title_filename='balse/works.csv')

In [4]:
from mangaki.utils.balse import MangakiBALSE
from mangaki.settings import DATA_DIR
from scipy.sparse import load_npz
import os.path

T = load_npz(os.path.join(DATA_DIR, 'balse/tag-matrix.npz')).toarray()

In [5]:
from sklearn.model_selection import ShuffleSplit

NB_SPLIT = 5
k_fold = ShuffleSplit(n_splits=NB_SPLIT)
SETS = list(k_fold.split(dataset.anonymized.X))

In [6]:
GOD_I = 0

In [7]:
# START
i_train, i_test = SETS[GOD_I]

In [45]:
NOTVAL = round(0.7 * len(i_train))

X_train = dataset.anonymized.X[i_train]
y_train = dataset.anonymized.y[i_train]
X_subtrain = X_train[:NOTVAL]
y_subtrain = y_train[:NOTVAL]
X_val = X_train[NOTVAL:]
y_val = y_train[NOTVAL:]
X_test = dataset.anonymized.X[i_test]
y_test = dataset.anonymized.y[i_test]
nb_train_rated = Counter(X_train[:, 1])

In [9]:
def run_als(X_train, X_test, y_train, y_test):
    als = MangakiALS(10, LAMBDA=0.1)
    als.set_parameters(dataset.anonymized.nb_users, dataset.anonymized.nb_works)
    als.fit(X_train, y_train)
    #als.all_errors(X_train, X_test, y_train, y_test)
    return als

In [10]:
als = run_als(X_train, X_test, y_train, y_test)
sub_als = run_als(X_subtrain, X_test, y_subtrain, y_test)

Computing M: (2079 × 9979)
Chrono: fill and center matrix [0q, 1823ms]
Shapes (2079, 10) (10, 9979)
Chrono: factor matrix [0q, 6365ms]
Computing M: (2079 × 9979)
Chrono: fill and center matrix [0q, 1179ms]
Shapes (2079, 10) (10, 9979)
Chrono: factor matrix [0q, 5401ms]


In [94]:
from mangaki.utils.lasso import MangakiLASSO

def run_lasso(X_train, X_test, y_train, y_test):
    lasso = MangakiLASSO(10, LAMBDA=0.01)
    lasso.load_tags(T)
    lasso.set_parameters(dataset.anonymized.nb_users, dataset.anonymized.nb_works)
    # lasso.fit(X_train, y_train)
    lasso.all_errors(X_train, X_test, y_train, y_test)
    return lasso

In [95]:
lasso = run_lasso(X_train, X_test, y_train, y_test)
sub_lasso = run_lasso(X_subtrain, X_test, y_subtrain, y_test)

0


/Users/jilljenn/code/mangaki/venv/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


500
1000
1500
2000
Train error 1.12071984201
1054 not
1054 not
1040 not
2004 not
1532 not
890 not
1406 not
782 not
687 not
Test error 1.28351969054
0
500
1000
1500
2000
Train error 1.06632774902
1054 not
1054 not
1040 not
2004 not
1532 not
890 not
1406 not
782 not
1677 not
687 not
1009 not
Test error 1.30442668241


In [96]:
i_test_ordered = sorted(i_test, key=lambda i: nb_train_rated[dataset.anonymized.X[i][1]], reverse=True)

In [97]:
X_tmp_test = dataset.anonymized.X[i_test_ordered]
y_tmp_test = dataset.anonymized.y[i_test_ordered]

In [98]:
# Attention
# X_val = X_train

In [99]:
nb_r = list(map(lambda x: nb_train_rated[x[1]], X_val))

In [110]:
y_val_als = sub_als.predict(X_val)
y_val_lasso = sub_lasso.predict(X_val)

754 not
702 not
1700 not
239 not
1765 not
706 not
1161 not
265 not
689 not
887 not
521 not
689 not
2002 not
1009 not
521 not
1396 not
1263 not
697 not
670 not
954 not
1677 not
1001 not
1888 not
104 not
1396 not
703 not
697 not
1460 not


In [111]:
y_als = sub_als.predict(X_tmp_test)
y_lasso = sub_lasso.predict(X_tmp_test)

1054 not
890 not
687 not
1040 not
1054 not
2004 not
1009 not
1406 not
1532 not
1677 not
782 not


In [112]:
import numpy as np

X = np.column_stack((nb_r, y_val_als, y_val_lasso))
y = y_val

In [113]:
X.shape

(90285, 3)

In [114]:
y_val_als.shape

(90285,)

In [115]:
y.shape

(90285,)

In [116]:
import tensorflow as tf

In [117]:
beta = tf.Variable(tf.random_normal([1]), name='beta')
gamma = tf.Variable(tf.random_normal([1]), name='gamma')

In [118]:
e1 = np.array([1, 0, 0])
e2 = np.array([0, 1, 0])
e3 = np.array([0, 0, 1])
pred = tf.sigmoid(beta*(X.dot(e1) - gamma)) * X.dot(e2) + (1 - tf.sigmoid(beta*(X.dot(e1) - gamma))) * X.dot(e3)
loss = tf.reduce_mean(tf.square(y - pred)) ** 0.5
reg_loss = loss

global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 0.9
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           20, 0.9965402628278678, staircase=True)
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_step = optimizer.minimize(reg_loss, var_list=[beta, gamma], global_step=global_step)

init_op = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init_op)

In [122]:
for i in range(5000):
    sess.run(train_step)
    if i % 500 == 0:
        print('loss', sess.run(reg_loss), 'beta', beta.eval(), 'gamma', gamma.eval(), learning_rate.eval())

loss 1.18477 beta [ 5.49957323] gamma [ 1.31185508] 0.00497174
loss 1.18477 beta [ 5.49957323] gamma [ 1.31219471] 0.00418071
loss 1.18477 beta [ 5.49957323] gamma [ 1.31248426] 0.00351554
loss 1.18477 beta [ 5.49957323] gamma [ 1.31272268] 0.00295621
loss 1.18477 beta [ 5.49957323] gamma [ 1.31291926] 0.00248586
loss 1.18477 beta [ 5.49957323] gamma [ 1.31309807] 0.00209035
loss 1.18477 beta [ 5.49957323] gamma [ 1.31323147] 0.00175777
loss 1.18477 beta [ 5.49957323] gamma [ 1.31335068] 0.0014781
loss 1.18477 beta [ 5.49957323] gamma [ 1.31346989] 0.00124293
loss 1.18477 beta [ 5.49957323] gamma [ 1.3135401] 0.00104517


In [123]:
nb_rt = list(map(lambda x: nb_train_rated[x[1]], X_tmp_test))

In [124]:
Xt = np.column_stack((nb_rt, y_als, y_lasso))

In [125]:
loss.eval(), beta.eval(), gamma.eval()

(1.1847682,
 array([ 5.49957323], dtype=float32),
 array([ 1.31359959], dtype=float32))

In [126]:
y_full_als = als.predict(X_tmp_test)
y_full_lasso = lasso.predict(X_tmp_test)

1054 not
890 not
687 not
1040 not
1054 not
2004 not
1406 not
1532 not
782 not


In [127]:
new_pred = tf.sigmoid(beta*(Xt.dot(e1) - gamma)) * y_full_als + (1 - tf.sigmoid(beta*(Xt.dot(e1) - gamma))) * y_full_lasso

In [129]:
WINDOW = 1000000

rmse_als_full = als.compute_rmse(y_full_als, y_tmp_test)
print('Test error', rmse_als_full)
god_rmse['als'].append(rmse_als_full)

Test error 1.15664171893


In [130]:
y_new_pred = new_pred.eval()
rmse_balse_full = als.compute_rmse(y_new_pred, y_tmp_test)
print('Test error', rmse_balse_full)
god_rmse['balse'].append(rmse_balse_full)

Test error 1.15330920105


In [131]:
rmse_lasso_full = als.compute_rmse(y_full_lasso, y_tmp_test)
print('Test error', rmse_lasso_full)
god_rmse['lasso'].append(rmse_lasso_full)

Test error 1.28351969054


In [132]:
for WINDOW, tag in [(1000, 'cold'), (len(nb_rt) - nb_rt.index(0), 'freeze')]:
    god_rmse['als-%s' % tag].append(als.compute_rmse(y_als[-WINDOW:], y_tmp_test[-WINDOW:]))
    y_new_pred = new_pred.eval()
    god_rmse['balse-%s' % tag].append(als.compute_rmse(y_new_pred[-WINDOW:], y_tmp_test[-WINDOW:]))
    god_rmse['lasso-%s' % tag].append(als.compute_rmse(y_lasso[-WINDOW:], y_tmp_test[-WINDOW:]))

In [133]:
import math

def avgstd(l):  # Displays mean and variance
    n = len(l)
    mean = float(sum(l)) / n
    var = float(sum(i * i for i in l)) / n - mean * mean
    return '%.5f ± %.5f' % (round(mean, 5), round(1.96 * math.sqrt(var / n), 3))  # '%.3f ± %.3f' % 

In [134]:
for key in god_rmse:
    print(key, god_rmse[key])
    print(key, avgstd(god_rmse[key]))

als [1.1566417189342415, 1.1566417189342415, 1.1566417189342415, 1.1566417189342415]
als 1.15664 ± 0.00000
balse [1.1566364570395711, 1.1557009140671186, 1.1533092010505441]
balse 1.15522 ± 0.00200
lasso [1.2683332115531472, 1.2683332115531472, 1.2835196905412267]
lasso 1.27340 ± 0.00800
als-cold [1.3298691481100322, 1.3298691481100322, 1.3298691481100322]
als-cold 1.32987 ± 0.00000
balse-cold [1.3273417687808244, 1.2998301376728705, 1.2267183338476031]
balse-cold 1.28463 ± 0.04800
lasso-cold [1.2355318961990578, 1.2355318961990578, 1.2265554514742143]
lasso-cold 1.23254 ± 0.00500
als-freeze [1.5003995376065826, 1.5003995376065826, 1.5003995376065826]
als-freeze 1.50040 ± 0.00000
balse-freeze [1.6291903972929971, 1.5375026795898905, 1.2970251892632048]
balse-freeze 1.48791 ± 0.15900
lasso-freeze [1.3018808470593315, 1.3018808470593315, 1.286085569935399]
lasso-freeze 1.29662 ± 0.00800


In [135]:
print(WINDOW)
for i in range(1, 4):
    encoded_user_id, encoded_work_id = X_tmp_test[-i]
    if True:#encoded_user_id == 2015:
        print(X_tmp_test[-i])
        print(dataset.titles[encoded_work_id],
              'als', als.predict(np.array([(encoded_user_id, encoded_work_id)])),
              'lasso', lasso.predict(np.array([(encoded_user_id, encoded_work_id)])),
              'sub_als', y_als[-i],
              'sub_lasso', y_lasso[-i],
              'balse', y_new_pred[-i],
              'truth', y_tmp_test[-i],
              'user bias', als.means[encoded_user_id],
              'nb ratings', nb_train_rated[encoded_work_id])

240
[251 3013]
Power Stone als [ 0.74640613] lasso [0.84433903611047678] sub_als 0.584779322389 sub_lasso 0.831232580721 balse 0.844268 truth -2.0 user bias 0.809893993523 nb ratings 0
[389 1635]
Lingeries als [ 1.71536751] lasso [0.5061941207916113] sub_als 1.28279391517 sub_lasso 0.417444647805 balse 0.507075 truth 2.0 user bias 0.547940075342 nb ratings 0
[1306 4586]
The pornography novelist is trained als [ 1.31399977] lasso [0.82496147941076636] sub_als 0.45627384299 sub_lasso 0.811991279273 balse 0.825318 truth 0.1 user bias 0.824961479411 nb ratings 0


In [137]:
with open(os.path.join(DATA_DIR, 'balse/balse.csv')) as f:
    tags = [line.split(',')[1] for line in f]

WORK_ID = 665
    
for tag, weight in zip(tags, T[WORK_ID]):
    if weight != 0:
        print(tag.strip(), weight)

2girls 0.173580855131
tree 0.163075566292
1girl 0.110314182937
brown hair 0.330881863832
long hair 0.413392990828
short hair 0.279685050249
serafuku 0.170303434134
thighhighs 0.256292790174
skirt 0.431603550911
1boy 0.149014562368
outdoors 0.118006154895
bag 0.100744493306
2boys 0.130534887314
multiple girls 0.743197321892
3girls 0.455147713423
smile 0.164101824164
cloud 0.186545163393
black hair 0.282055705786
brown eyes 0.317968964577
twintails 0.145467013121
multiple boys 0.195584014058
pleated skirt 0.143671616912
sky 0.223109439015
school uniform 0.79898160696


In [138]:
for tag, weight in zip(tags, lasso.reg[2015].coef_):
    if weight != 0:
        print(tag, weight)

In [139]:
lasso.reg[2015].predict(np.array([T[WORK_ID]]))

array([ 0.73747573])

In [140]:
lasso.reg[2015].coef_.dot(T[WORK_ID])

0.0

In [141]:
lasso.reg[2015].intercept_

0.73747572817848728

In [142]:
for tag, weight in zip(tags, lasso.reg[2015].coef_ * T[WORK_ID]):
    if weight:
        print(tag.strip(), weight)

In [143]:
als.VT.T[3748]

array([-0.34922168, -0.02603758, -0.31870448,  0.37174495, -0.04622742,
        1.42612367, -0.84433803,  0.20316341,  0.80450289,  1.12107641])

In [144]:
from collections import Counter
Counter(User.objects.get(id=1).rating_set.values_list('choice', flat=True))

Counter({'dislike': 16,
         'favorite': 23,
         'like': 118,
         'neutral': 9,
         'willsee': 330,
         'wontsee': 68})

In [145]:
for work_id, title in enumerate(dataset.titles):
    if title == 'Millennium Actress':
        print(work_id)

77
9163


In [146]:
from collections import Counter, defaultdict

nb = defaultdict(Counter)
for (user_id, work_id), value in zip(dataset.anonymized.X, dataset.anonymized.y):
    nb[user_id][value] += 1

In [147]:
for (user_id, work_id), value in zip(dataset.anonymized.X, dataset.anonymized.y):
    if work_id == 77 and value == 4:
        print(user_id, nb[user_id])

981 Counter({-0.5: 126, 2.0: 92, -2.0: 51, 4.0: 46, 0.5: 42, 0.1: 28})
121 Counter({-0.5: 75, 0.5: 64, 4.0: 62, 2.0: 51, 0.1: 22, -2.0: 5})
1738 Counter({4.0: 21, -2.0: 8, 0.1: 7, 2.0: 7, 0.5: 4, -0.5: 1})
1120 Counter({0.5: 742, 2.0: 316, -0.5: 208, -2.0: 137, 0.1: 126, 4.0: 39})
326 Counter({2.0: 123, 4.0: 70, 0.1: 55, -2.0: 46, 0.5: 25, -0.5: 11})
1689 Counter({0.1: 71, -2.0: 43, 2.0: 42, 4.0: 7})
1979 Counter({2.0: 61, 0.5: 53, -0.5: 34, 4.0: 31, 0.1: 14})
278 Counter({2.0: 103, -0.5: 85, 0.5: 31, 4.0: 26, 0.1: 24, -2.0: 10})
407 Counter({0.1: 13, -2.0: 11, -0.5: 9, 2.0: 9, 0.5: 3, 4.0: 1})
512 Counter({4.0: 7, 2.0: 6, -2.0: 2, -0.5: 2, 0.5: 1})
453 Counter({-0.5: 518, 0.5: 437, 2.0: 278, 4.0: 77, 0.1: 17, -2.0: 11})
2015 Counter({0.5: 333, 2.0: 121, -0.5: 72, 4.0: 25, -2.0: 16, 0.1: 9})
1839 Counter({2.0: 197, -0.5: 73, 4.0: 62, 0.5: 57, 0.1: 46, -2.0: 16})
242 Counter({2.0: 199, 4.0: 37, 0.1: 34, -2.0: 26, -0.5: 4, 0.5: 4})
1678 Counter({2.0: 6, 0.5: 5, 4.0: 5, -2.0: 2})


In [148]:
# JJ est 2015 !